# Initial setup

In [ ]:
!pip install pytorch-pretrained-bert --quiet
!pip install -qq transformers

     |████████████████████████████████| 123 kB 9.6 MB/s 
     |████████████████████████████████| 132 kB 38.9 MB/s 
     |████████████████████████████████| 79 kB 2.9 MB/s 
     |████████████████████████████████| 8.7 MB 43.2 MB/s 
     |████████████████████████████████| 138 kB 46.3 MB/s 
     |████████████████████████████████| 127 kB 21.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 6.9 MB/s 
     |████████████████████████████████| 596 kB 25.3 MB/s 
     |████████████████████████████████| 77 kB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 40.4 MB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 


In [ ]:
import torch
# from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence


# Connect to my Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# First attempt - Useless

In [ ]:
def tokenize(marked_text, tokenizer):
    tokenized_text = tokenizer.tokenize(marked_text)
    segments_ids = [1] * len(tokenized_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    return tokenized_text, segments_ids, indexed_tokens
 

In [ ]:
def generate_token_embeddings(model, indexed_tokens, segments_ids):
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    model.eval()
    # Predict hidden states features for each layer
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors)

    # Concatenate the tensors for all layers. We use `stack` here to
    # create a new dimension in the tensor.
    token_embeddings = torch.stack(encoded_layers, dim=0)

    # Remove dimension 1, the "batches".
    token_embeddings = torch.squeeze(token_embeddings, dim=1)

    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)

    token_embeddings.size()

    # Stores the token vectors, with shape [23 x 768]
    token_vecs_sum = []

    # `token_embeddings` is a [23 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor
        # Sum the vectors from the last four layers.
        sum_vec = torch.sum(token[-4:], dim=0)
        
        # Use `sum_vec` to represent `token`.
        token_vecs_sum.append(sum_vec)
    return token_vecs_sum



In [ ]:
def generate_sentence_embeddings(model, indexed_tokens, segments_ids):
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    model.eval()
    # Predict hidden states features for each layer
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors)

    # `token_vecs` is a tensor with shape [23 x 768]
    token_vecs = encoded_layers[11][0]

    # Calculate the average of all 23 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
    
    return sentence_embedding



In [ ]:
def split_corpus(corpus):
    """ Split the corpus into multiple sentences and mark them with [CLS] and [SEP] tags.
    Returns:
    marked_sentences: 
    lengths: 
    """

    corpus = " " + corpus
    sentences = corpus.split('<eos>')[:-1]
    marked_sentences = ["[CLS]" + s + "[SEP]" for s in sentences]
    lengths = [len(s) for s in marked_sentences]
    return marked_sentences, lengths

def tokenize_corpus(bert_model, bert_tokenizer, marked_sentences):
    word_embeddings = []
    sentence_embeddings = []
    for sentence in marked_sentences:

        tokenized_text, segments_ids, indexed_tokens = tokenize(sentence, bert_tokenizer)
        token_vecs_sum = generate_token_embeddings(bert_model, indexed_tokens, segments_ids)
        sentence_embedding = generate_sentence_embeddings(bert_model, indexed_tokens, segments_ids)

        # print('Sentence: {}', sentence)
        # print('Number of tokens: {} Shape of each token embedding: {}'.format(len(token_vecs_sum), token_vecs_sum[0].shape))
        # print('Shape of each sentence embedding: {}\n'.format(sentence_embedding.shape))

        word_embeddings.append(token_vecs_sum)
        sentence_embeddings.append(sentence_embedding)

    return word_embeddings, sentence_embeddings


In [ ]:
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [ ]:
corpus =  "Disposing of certain hazardous wastes without necessary authorization and without compliance with regulatory requirements <eos> the complaint seeks a court order directing navajo to comply with these regulatory standards and civil penalties for the alleged non-compliance navajo has answered the complaint <eos>"
marked_sentences, lengths = split_corpus(corpus)
word_embeddings, sentence_embeddings = tokenize_corpus(bert_model, bert_tokenizer, marked_sentences)

TypeError: ignored

In [ ]:
print(len(sentence_embeddings), len(sentence_embeddings[0]), len(sentence_embeddings[1]))

NameError: ignored

# Split dataset

In [ ]:

root = '/content/drive/MyDrive/11785_Project/Programming/Data/fiqa2018/'
df1 = pd.read_csv(f"{root}FiQA_ABSA_task1/task1_headline_ABSA_train.csv")
df2 = pd.read_csv(f"{root}FiQA_ABSA_task1/task1_post_ABSA_train.csv")
df = pd.concat([df1, df2])

df_test1 = pd.read_csv(f"{root}FIQA_ABSA_task1_test/task1_post_ABSA_test.csv")
df_test2 = pd.read_csv(f"{root}FIQA_ABSA_task1_test/task1_headline_ABSA_test.csv")
df_test = pd.concat([df_test1, df_test2])



FileNotFoundError: ignored

In [ ]:
# Uncomment for classification task
# df["sentiment_score"] = df["sentiment_score"].apply(lambda x: 1 if x>0 else 0)


In [ ]:
df_train, df_val = train_test_split(df, test_size = 0.18, random_state = 10)
len(df_train), len(df_val), len(df_test)


(911, 200, 192)

In [ ]:
df_train.head()

,Unnamed: 0,id,sentence,snippets,target,sentiment_score,aspects
363,363,16692,NaturalGas Settles At 3-year Low $DBO $BNO,Settles At 3-year Low,DBO,-0.563,Stock/Price Action/Bearish
491,491,17562,$FB & $TSLA cracking lower early. $short #corr...,cracking lower earl,TSLA,-0.511,Stock/Price Action/Bearish/Bearish Behavior
151,151,539,Sainsbury sales slip again as price pressures ...,pressures take a toll,Sainsbury,-0.303,Stock/Price Action
656,656,19015,$JD down 20% in after-hour market. anyone care...,down 20% in after-hour market,JD,-0.551,Stock/Price Action/Bearish/Bearish Behavior
520,520,17831,"$FB trend, hits mid 110's and drops",hits mid 110's and drops,FB,-0.343,Stock/Price Action/Bearish/Bearish Behavior


## Analysing data (to get max length for padding)

In [ ]:
sentences = df.sentence.to_list()
sentiment_scores = torch.tensor(df.sentiment_score.to_list())
print(sentences[:2], sentiment_scores[:2])

['Royal Mail chairman Donald Brydon set to step down', 'Stakes High for AstraZeneca Heart Drug Facing Tough Competition'] tensor([-0.3740, -0.2400])


In [ ]:
token_lens = [len(bert_tokenizer.encode(sentence)) for sentence in sentences]
max_length = max(token_lens)
max_length = int(np.power(2, np.ceil(np.log2(max_length)))) #  Raise maxlength to the next power of 2
max_length

NameError: ignored

In [ ]:
sample_sentence = sentences[0]
sample_sentence

'Royal Mail chairman Donald Brydon set to step down'

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
sentence_encoding = bert_tokenizer.encode_plus(sample_sentence, return_tensors = 'pt', max_length = 1024) # Return encoding as pytorch tensor
input_ids, attention_mask = sentence_encoding['input_ids'], sentence_encoding['attention_mask']
(input_ids)

tensor([[  101,  1787, 11508,  3931,  5554,   139,  1616,  3842,  1383,  1106,
          2585,  1205,   102]])

# SentimentLinear - BERT

## Dataloader

In [ ]:
class FiQaSamples(torch.utils.data.Dataset):

  def __init__(self, partition, sentences, sentiment_scores, tokenizer, max_len):
    self.sentences = sentences
    self.sentiment_scores = sentiment_scores
    assert len(sentences) == len(sentiment_scores)

    self.tokenizer = tokenizer
    self.length = len(self.sentences)
    self.max_len = max_len
  
  def __len__(self):
    return self.length
  
  def __getitem__(self, index):
    sentence = self.sentences[index]
    score = self.sentiment_scores[index]

    encoding = self.tokenizer.encode_plus(
      sentence,
      max_length=self.max_len,
      padding='max_length',
      return_tensors='pt',
    )

    return encoding['input_ids'].flatten(), encoding['attention_mask'].flatten(), torch.tensor(score, dtype = torch.float)

In [ ]:
batch_size = 32
train_sentences, train_scores = df_train.sentence.to_list(), df_train.sentiment_score.to_list()
val_sentences, val_scores = df_val.sentence.to_list(), df_val.sentiment_score.to_list()

train_data = FiQaSamples('train', train_sentences, train_scores, bert_tokenizer, 128)
val_data = FiQaSamples('dev', val_sentences, val_scores, bert_tokenizer, 128)
# test_data = FiQaSamples('test_order.csv')

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=1) # TODO: Define the train loader. Remember to pass in a parameter (function) for the collate_fn argument 
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=1,) # TODO: Define the val loader. Remember to pass in a parameter (function) for the collate_fn argument 
# test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=1, collate_fn=LibriSamplesTest.collate_fn ) # TODO: Define the test loader. Remember to pass in a parameter (function) for the collate_fn argument 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Batch size: ", batch_size)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
# print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))
print("Device: ", device)

Batch size:  32
Train dataset samples = 911, batches = 29
Val dataset samples = 200, batches = 7
Device:  cuda


## Model Architecture

In [ ]:
# https://srinivas-yeeda.medium.com/sentiment-analysis-using-word2vec-and-glove-embeddings-5ad7d50ddb0d
class SentimentClassifier(nn.Module):

  def __init__(self):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-cased')
    self.drop = nn.Dropout(p=0.3)
    self.out1 = nn.Linear(self.bert.config.hidden_size, 2048)
    self.out2 = nn.Linear(2048, 1024)
    self.out3 = nn.Linear(1024, 1)
    self.gelu = nn.GELU()
    self.sigmoid = nn.Sigmoid() # TODO: Remove
  
  def forward(self, input_ids, attention_mask):
    output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )[1]
    output = self.out1(output)
    output = self.gelu(output)
    output = self.out2(output)
    output = self.gelu(output)
    output = self.out3(output)
    return output


## Training and Validation functions

In [ ]:
def run_train(model, optimizer, criterion, scheduler, train_loader):
    torch.cuda.empty_cache()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    model.train()    
    total_loss = 0

    total_ele = 0
    for i, (input_ids, attention_mask, y) in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        y = y.to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
          )
        
        outputs = outputs.squeeze() # TODO: May have to remove this
        loss = criterion(outputs, y)

        total_loss += float(loss.item()) 
        total_ele += int(len(y))
        batch_bar.set_postfix(
          loss="{:.04f}".format(float(total_loss / (i + 1))),
          lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])),
          total_ele="{}".format(int(total_ele))
          )
        
        loss.backward()
        optimizer.step()
        batch_bar.update() # Update tqdm bar
        
    scheduler.step()
    batch_bar.close() # You need this to close the tqdm bar
    
    print("Train Loss: {}".format(total_loss / len(train_loader)))

def run_validation(model, val_loader):
    torch.cuda.empty_cache()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    model.eval()    
    correct_pred = 0
    total_loss = 0

    total_ele = 0
    with torch.no_grad():
        for i, (input_ids, attention_mask, y) in enumerate(val_loader):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            y = y.to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
              )
            
            outputs = outputs.squeeze() # TODO: May have to remove this
            loss = criterion(outputs, y)
            total_loss += float(loss.item()) 

            batch_bar.set_postfix(
              lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])),
              )
            
            batch_bar.update() # Update tqdm bar
            
        batch_bar.close() # You need this to close the tqdm bar
        
        print("Val Loss: {}".format(total_loss / len(val_loader)))

In [ ]:
lr = 1e-3
epochs = 10

model = SentimentClassifier()
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
criterion = nn.MSELoss()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
for epoch in range(epochs):
    run_train(model, optimizer, criterion, scheduler, train_loader)
    run_validation(model, val_loader)


Val Loss: 0.18358829191752843


Train Loss: 1.4068537359607631


Val Loss: 0.17013397174222128


Train Loss: 0.16777667865670962


Val Loss: 0.16994145938328334


...

Train Loss: 0.160950122722264


Val Loss: 0.16999431167330062


Train Loss: 0.1605138049043458


Val Loss: 0.16998342956815446


Train Loss: 0.16087729766451078


# SentimentRNN - BERT

In [ ]:
BERT_EMBEDDING_SIZE = 768
bert_model = BertModel.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Dataloader

In [ ]:
class FiQaSamplesBertWord(torch.utils.data.Dataset):

  def __init__(self, partition, sentences, sentiment_scores, tokenizer, max_len):
    self.sentences = sentences
    self.sentiment_scores = sentiment_scores
    assert len(sentences) == len(sentiment_scores)

    self.tokenizer = tokenizer
    self.length = len(self.sentences)
    self.max_len = max_len
  
  def __len__(self):
    return self.length
  
  def __getitem__(self, index):
    sentence = self.sentences[index]
    score = self.sentiment_scores[index]

    encoding = self.tokenizer.encode_plus(
      sentence,
      max_length=self.max_len,
      padding='max_length',
      return_tensors='pt',
    )

    return encoding['input_ids'].flatten(), encoding['attention_mask'].flatten(), torch.tensor(score, dtype = torch.float)

In [ ]:
batch_size = 32
train_sentences, train_scores = df_train.sentence.to_list(), df_train.sentiment_score.to_list()
val_sentences, val_scores = df_val.sentence.to_list(), df_val.sentiment_score.to_list()

train_data = FiQaSamplesBertWord('train', train_sentences, train_scores, bert_tokenizer, 128)
val_data = FiQaSamplesBertWord('dev', val_sentences, val_scores, bert_tokenizer, 128)
# test_data = FiQaSamplesBertWord('test_order.csv')

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=1) # TODO: Define the train loader. Remember to pass in a parameter (function) for the collate_fn argument 
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=1,) # TODO: Define the val loader. Remember to pass in a parameter (function) for the collate_fn argument 
# test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=1, collate_fn=LibriSamplesTest.collate_fn ) # TODO: Define the test loader. Remember to pass in a parameter (function) for the collate_fn argument 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Batch size: ", batch_size)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
# print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))
print("Device: ", device)

Batch size:  32
Train dataset samples = 911, batches = 29
Val dataset samples = 200, batches = 7
Device:  cuda


## Model Architecture

In [ ]:
# https://www.kaggle.com/code/arunmohan003/sentiment-analysis-using-lstm-pytorch/notebook
class SentimentBertRNN(nn.Module):
    def __init__(self):
        super(SentimentBertRNN, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.hidden_size = 128
        self.lstm = nn.LSTM(input_size=BERT_EMBEDDING_SIZE, hidden_size=self.hidden_size, num_layers=2, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, 1)
        
    def forward(self, input_ids, attention_mask):
        # x is a list of vector (num of tokens, emb_dim - hyperparam)
        batch_size = input_ids.shape[0]
        out = self.bert(input_ids = input_ids, attention_mask = attention_mask)[0]
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.dropout(out)
        out = self.fc(out)

        out = out.view(batch_size, -1)
        out = out[:, -1] # get last batch of labels
        return out

## Training and validation functions

In [ ]:
def run_train(model, optimizer, criterion, scheduler, train_loader):
    torch.cuda.empty_cache()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    model.train()    
    total_loss = 0

    total_ele = 0
    for i, (input_ids, attention_mask, y) in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        y = y.to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
          )
        
        outputs = outputs.squeeze() # TODO: May have to remove this
        loss = criterion(outputs, y)

        total_loss += float(loss.item()) 
        total_ele += int(len(y))
        batch_bar.set_postfix(
          loss="{:.04f}".format(float(total_loss / (i + 1))),
          lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])),
          total_ele="{}".format(int(total_ele))
          )
        
        loss.backward()
        optimizer.step()
        batch_bar.update() # Update tqdm bar
        
    scheduler.step()
    batch_bar.close() # You need this to close the tqdm bar
    
    print("Train Loss: {}".format(total_loss / len(train_loader)))

def run_validation(model, val_loader):
    torch.cuda.empty_cache()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    model.eval()    
    correct_pred = 0
    total_loss = 0

    total_ele = 0
    with torch.no_grad():
        for i, (input_ids, attention_mask, y) in enumerate(val_loader):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            y = y.to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
              )
            
            outputs = outputs.squeeze() # TODO: May have to remove this
            loss = criterion(outputs, y)
            total_loss += float(loss.item()) 

            batch_bar.set_postfix(
              lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])),
              )
            
            batch_bar.update() # Update tqdm bar
            
        batch_bar.close() # You need this to close the tqdm bar
        
        print("Val Loss: {}".format(total_loss / len(val_loader)))

In [ ]:
lr = 1e-3
epochs = 10

model = SentimentBertRNN()
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
criterion = nn.MSELoss()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Lets train!

In [ ]:
for epoch in range(epochs):
    run_train(model, optimizer, criterion, scheduler, train_loader)
    run_validation(model, val_loader)


Train Loss: 1.8752116763900066


Val Loss: 0.17607808538845607


Train Loss: 0.18389656677328306


Val Loss: 0.17218721125807082


Train Loss: 0.18046144164841751


...

Val Loss: 0.16994784346648625


Train Loss: 0.16396602397334986


Val Loss: 0.17066440199102675


Train Loss: 0.16824036682474203


Val Loss: 0.1702498346567154


# SentimentRNN - Word2Vec

In [ ]:

EMBED_DIMENSION = 300
EMBED_MAX_NORM = 1

## Some necessary functions

In [ ]:
# Models

class CBOW_Model(nn.Module):
    """
    Implementation of CBOW model described in paper:
    https://arxiv.org/abs/1301.3781
    """
    def __init__(self, vocab_size: int):
        super(CBOW_Model, self).__init__()
        self.embeddings = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=EMBED_DIMENSION,
            max_norm=EMBED_MAX_NORM,
        )
        self.linear = nn.Linear(
            in_features=EMBED_DIMENSION,
            out_features=vocab_size,
        )

    def forward(self, inputs_):
        x = self.embeddings(inputs_)
        x = x.mean(axis=1)
        x = self.linear(x)
        return x


class SkipGram_Model(nn.Module):
    """
    Implementation of Skip-Gram model described in paper:
    https://arxiv.org/abs/1301.3781
    """
    def __init__(self, vocab_size: int):
        super(SkipGram_Model, self).__init__()
        self.embeddings = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=EMBED_DIMENSION,
            max_norm=EMBED_MAX_NORM,
        )
        self.linear = nn.Linear(
            in_features=EMBED_DIMENSION,
            out_features=vocab_size,
        )

    def forward(self, inputs_):
        # print(f'input shape: {inputs_.shape}')
        # print(f'input: {inputs_}')
        x = self.embeddings(inputs_)
        x = self.linear(x)
        return x

## Word2Vec Utility Function

In [ ]:
word2vec_dir = '/content/drive/MyDrive/11785_Project/Programming/Data/Project/word2vec/'
# vocab = torch.load(f'{word2vec_dir}vocab.pt')
# model = torch.load(f'{word2vec_dir}model.pt')

# # embedding from first model layer
# embeddings = list(model.parameters())[0]
# embeddings = embeddings.cpu().detach().numpy()

# # normalization
# norms = (embeddings * 2).sum(axis=1) * (1 / 2)
# norms = np.reshape(norms, (len(norms), 1))
# embeddings_norm = embeddings / norms
# embeddings_norm.shape

# def get_word_embedding(word: str):
#     word_id = vocab[word]
#     if word_id == 0:
#         # print("Out of vocabulary word")
#         return torch.zeros(embeddings_norm.shape[1], 1)

#     word_vec = embeddings_norm[word_id]
#     word_vec = np.reshape(word_vec, (len(word_vec), 1))
#     return word_vec

# for w in ['burden', 'deliverable', 'domain']:
#     embedding = get_word_embedding(w)

#     print(f'{w} has embedding {embedding.shape}')

## Dataloader

In [ ]:
class FiQaSamplesWord2Vec(torch.utils.data.Dataset):

  def __init__(self, partition, sentences, sentiment_scores, get_word_embedding, max_len):
    self.sentences = sentences
    self.sentiment_scores = sentiment_scores
    self.model = get_word_embedding
    assert len(sentences) == len(sentiment_scores)
    self.length = len(self.sentences)
    self.max_len = max_len
  
  def __len__(self):
    return self.length
  
  def __getitem__(self, index):
    sentence = self.sentences[index]
    score = self.sentiment_scores[index]
    tokens = sentence
    # Tokens is a list of strings
    embedding = np.array([np.array(self.model(token)) for token in sentence.split()])
    return torch.tensor(embedding).squeeze(), torch.tensor(score, dtype = torch.float)

  def collate_fn(batch):
    batch_x = [emb for emb, y in batch]
    batch_y = [y for emb, y in batch]

    batch_x_pad = pad_sequence(batch_x, batch_first = True) 
    lengths_x = [x.shape[0] for x in batch_x]

    return batch_x_pad, lengths_x, torch.tensor(batch_y)

In [ ]:
# batch_size = 32
# train_sentences, train_scores = df_train.sentence.to_list(), df_train.sentiment_score.to_list()
# val_sentences, val_scores = df_val.sentence.to_list(), df_val.sentiment_score.to_list()

# train_data = FiQaSamplesWord2Vec('train', train_sentences, train_scores, get_word_embedding, 128)
# val_data = FiQaSamplesWord2Vec('dev', val_sentences, val_scores, get_word_embedding, 128)
# # test_data = FiQaSamples('test_order.csv')

# train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=1, collate_fn=FiQaSamplesWord2Vec.collate_fn) # TODO: Define the train loader. Remember to pass in a parameter (function) for the collate_fn argument 
# val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=1, collate_fn=FiQaSamplesWord2Vec.collate_fn) # TODO: Define the val loader. Remember to pass in a parameter (function) for the collate_fn argument 
# # test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=1, collate_fn=LibriSamplesTest.collate_fn ) # TODO: Define the test loader. Remember to pass in a parameter (function) for the collate_fn argument 

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# print("Batch size: ", batch_size)
# print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
# print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
# # print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))
# print("Device: ", device)

NameError: ignored

## Model Architecture

In [ ]:
# https://www.kaggle.com/code/arunmohan003/sentiment-analysis-using-lstm-pytorch/notebook
class SentimentRNN(nn.Module):
    def __init__(self, input_size):
        super(SentimentRNN,self).__init__()
        self.hidden_size = 128
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=self.hidden_size, num_layers=2, bidirectional = True, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size*2, 1)
        
    def forward(self, x, lx):
        # x is a list of vector (num of tokens, emb_dim - hyperparam)
        batch_size = x.shape[0]
        packed_input = pack_padded_sequence(x, lx, batch_first = True, enforce_sorted=False) 
        output, (hidden_state, cell_state) = self.lstm(packed_input) 
        out, lengths  = pad_packed_sequence(output, batch_first = True)
        out = out.contiguous().view(-1, self.hidden_size*2)
        out = self.dropout(out)
        out = self.fc(out)

        out = out.view(batch_size, -1)
        out = out[:, -1] # get last batch of labels
        return out, lengths


## Training and validation functions

In [ ]:
def run_train(model, train_loader, batch_size, optimizer, criterion, scheduler, epoch, epochs):
  torch.cuda.empty_cache()
  model.train()
  batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

  num_correct = 0
  total_loss = 0

  for i, (x, lx, y) in enumerate(train_loader):
      torch.cuda.empty_cache() # Use this often
      optimizer.zero_grad()

      x = x.to(device)
      y = y.to(device)

      outputs, lengths = model(x, lx)
      loss = criterion(outputs, y)
      total_loss += float(loss)
      batch_bar.set_postfix(
          loss="{:.04f}".format(float(total_loss / (i + 1))),
          lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
      
      # Another couple things you need for FP16. 
      loss.backward()
      optimizer.step()


      batch_bar.update() # Update tqdm bar
  scheduler.step() # We told scheduler T_max that we'd call step() (len(train_loader) * epochs) many times.
  batch_bar.close() # You need this to close the tqdm bar

  print("Epoch {}/{}: Train Loss {:.04f}, Learning Rate {:.04f}".format(
      epoch + 1,
      epochs,
      float(total_loss / len(train_loader)),
      float(optimizer.param_groups[0]['lr'])))
  


def run_validation(model, val_loader):
  torch.cuda.empty_cache() # Use this often
  model.eval()
  batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
  total_loss = 0
  with torch.no_grad():
      for i, (x, lx, y) in enumerate(val_loader):
          torch.cuda.empty_cache() # Use this often
          x = x.to(device)
          y = y.to(device)

          outputs, lengths = model(x, lx)
          loss = criterion(outputs, y)
          total_loss += loss
          batch_bar.set_postfix(dist="{:.04f}".format(total_loss / (i + 1)))

          batch_bar.update()
          
      batch_bar.close()


      print("Distance: {:.04f}".format(total_loss / len(val_loader)))

In [ ]:
lr = 1e-3
epochs = 10

## Lets train!

In [ ]:

for m in ['cbow', 'skipgram']:
    for s in [150, 300, 600]:
        print(f"embedding model: {m}_{s}")
        vocab = torch.load(f'{word2vec_dir}{m}_{s}/vocab.pt')
        embedding_model = torch.load(f'{word2vec_dir}{m}_{s}/model.pt')

        # embedding from first model layer
        embeddings = list(embedding_model.parameters())[0]
        embeddings = embeddings.cpu().detach().numpy()

        # normalization
        norms = (embeddings * 2).sum(axis=1) * (1 / 2)
        norms = np.reshape(norms, (len(norms), 1))
        embeddings_norm = embeddings / norms
        embeddings_norm.shape

        def get_word_embedding(word: str):
            word_id = vocab[word]
            if word_id == 0:
                # print("Out of vocabulary word")
                return torch.zeros(embeddings_norm.shape[1], 1)

            word_vec = embeddings_norm[word_id]
            word_vec = np.reshape(word_vec, (len(word_vec), 1))
            return word_vec

        batch_size = 32
        train_sentences, train_scores = df_train.sentence.to_list(), df_train.sentiment_score.to_list()
        val_sentences, val_scores = df_val.sentence.to_list(), df_val.sentiment_score.to_list()

        train_data = FiQaSamplesWord2Vec('train', train_sentences, train_scores, get_word_embedding, 128)
        val_data = FiQaSamplesWord2Vec('dev', val_sentences, val_scores, get_word_embedding, 128)
        # test_data = FiQaSamples('test_order.csv')

        train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=1, collate_fn=FiQaSamplesWord2Vec.collate_fn) # TODO: Define the train loader. Remember to pass in a parameter (function) for the collate_fn argument 
        val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=1, collate_fn=FiQaSamplesWord2Vec.collate_fn) # TODO: Define the val loader. Remember to pass in a parameter (function) for the collate_fn argument 
        # test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=1, collate_fn=LibriSamplesTest.collate_fn ) # TODO: Define the test loader. Remember to pass in a parameter (function) for the collate_fn argument 

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        model = SentimentRNN(s)
        model.to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
        criterion = nn.MSELoss()

        for epoch in range(epochs):
            run_train(model, train_loader, batch_size, optimizer, criterion, scheduler, epoch, epochs)
            run_validation(model, val_loader)


embedding model: cbow_150


Epoch 1/10: Train Loss 0.1652, Learning Rate 0.0010


Distance: 0.1742


Epoch 2/10: Train Loss 0.1628, Learning Rate 0.0009


Distance: 0.1705


...

Distance: 0.1705


Epoch 9/10: Train Loss 0.1573, Learning Rate 0.0000


Distance: 0.1718


Epoch 10/10: Train Loss 0.1582, Learning Rate 0.0000


Distance: 0.1721


In [ ]:
# embedding model: cbow_150
# Epoch 1/10: Train Loss 0.1819, Learning Rate 0.0010
# Distance: 0.1845
# Epoch 2/10: Train Loss 0.1750, Learning Rate 0.0009
# Distance: 0.1792
# Epoch 3/10: Train Loss 0.1712, Learning Rate 0.0008
# Distance: 0.1772
# Epoch 4/10: Train Loss 0.1673, Learning Rate 0.0007
# Distance: 0.1732
# Epoch 5/10: Train Loss 0.1662, Learning Rate 0.0005
# Distance: 0.1723
# Epoch 6/10: Train Loss 0.1645, Learning Rate 0.0003
# Distance: 0.1711
# Epoch 7/10: Train Loss 0.1645, Learning Rate 0.0002
# Distance: 0.1724
# Epoch 8/10: Train Loss 0.1621, Learning Rate 0.0001
# Distance: 0.1730
# Epoch 9/10: Train Loss 0.1636, Learning Rate 0.0000
# Distance: 0.1727
# Epoch 10/10: Train Loss 0.1619, Learning Rate 0.0000
# Distance: 0.1726
# embedding model: cbow_300
# Epoch 1/10: Train Loss 0.1785, Learning Rate 0.0010
# Distance: 0.1781
# Epoch 2/10: Train Loss 0.1724, Learning Rate 0.0009
# Distance: 0.1756
# Epoch 3/10: Train Loss 0.1681, Learning Rate 0.0008
# Distance: 0.1722
# Epoch 4/10: Train Loss 0.1657, Learning Rate 0.0007
# Distance: 0.1697
# Epoch 5/10: Train Loss 0.1647, Learning Rate 0.0005
# Distance: 0.1695
# Epoch 6/10: Train Loss 0.1629, Learning Rate 0.0003
# Distance: 0.1702
# Epoch 7/10: Train Loss 0.1601, Learning Rate 0.0002
# Distance: 0.1702
# Epoch 8/10: Train Loss 0.1632, Learning Rate 0.0001
# Distance: 0.1702
# Epoch 9/10: Train Loss 0.1613, Learning Rate 0.0000
# Distance: 0.1699
# Epoch 10/10: Train Loss 0.1596, Learning Rate 0.0000
# Distance: 0.1698
# embedding model: cbow_600
# Epoch 1/10: Train Loss 0.1974, Learning Rate 0.0010
# Distance: 0.1938
# Epoch 2/10: Train Loss 0.1878, Learning Rate 0.0009
# Distance: 0.1895
# Epoch 3/10: Train Loss 0.1800, Learning Rate 0.0008
# Distance: 0.1833
# Epoch 4/10: Train Loss 0.1764, Learning Rate 0.0007
# Distance: 0.1782
# Epoch 5/10: Train Loss 0.1709, Learning Rate 0.0005
# Distance: 0.1757
# Epoch 6/10: Train Loss 0.1687, Learning Rate 0.0003
# Distance: 0.1737
# Epoch 7/10: Train Loss 0.1685, Learning Rate 0.0002
# Distance: 0.1737
# Epoch 8/10: Train Loss 0.1657, Learning Rate 0.0001
# Distance: 0.1729
# Epoch 9/10: Train Loss 0.1654, Learning Rate 0.0000
# Distance: 0.1726
# Epoch 10/10: Train Loss 0.1658, Learning Rate 0.0000
# Distance: 0.1724
# embedding model: skipgram_150
# Epoch 1/10: Train Loss 0.1640, Learning Rate 0.0010
# Distance: 0.1714
# Epoch 2/10: Train Loss 0.1615, Learning Rate 0.0009
# Distance: 0.1709
# Epoch 3/10: Train Loss 0.1612, Learning Rate 0.0008
# Distance: 0.1700
# Epoch 4/10: Train Loss 0.1599, Learning Rate 0.0007
# Distance: 0.1704
# Epoch 5/10: Train Loss 0.1591, Learning Rate 0.0005
# Distance: 0.1692
# Epoch 6/10: Train Loss 0.1605, Learning Rate 0.0003
# Distance: 0.1698
# Epoch 7/10: Train Loss 0.1597, Learning Rate 0.0002
# Distance: 0.1690
# Epoch 8/10: Train Loss 0.1592, Learning Rate 0.0001
# Distance: 0.1699
# Epoch 9/10: Train Loss 0.1576, Learning Rate 0.0000
# Distance: 0.1701
# Epoch 10/10: Train Loss 0.1573, Learning Rate 0.0000
# Distance: 0.1702
# embedding model: skipgram_300
# Epoch 1/10: Train Loss 0.1912, Learning Rate 0.0010
# Distance: 0.1889
# Epoch 2/10: Train Loss 0.1827, Learning Rate 0.0009
# Distance: 0.1826
# Epoch 3/10: Train Loss 0.1770, Learning Rate 0.0008
# Distance: 0.1772
# Epoch 4/10: Train Loss 0.1725, Learning Rate 0.0007
# Distance: 0.1772
# Epoch 5/10: Train Loss 0.1681, Learning Rate 0.0005
# Distance: 0.1733
# Epoch 6/10: Train Loss 0.1670, Learning Rate 0.0003
# Distance: 0.1754
# Epoch 7/10: Train Loss 0.1656, Learning Rate 0.0002
# Distance: 0.1752
# Epoch 8/10: Train Loss 0.1636, Learning Rate 0.0001
# Distance: 0.1750
# Epoch 9/10: Train Loss 0.1639, Learning Rate 0.0000
# Distance: 0.1747
# Epoch 10/10: Train Loss 0.1623, Learning Rate 0.0000
# Distance: 0.1746
# embedding model: skipgram_600
# Epoch 1/10: Train Loss 0.1654, Learning Rate 0.0010
# Distance: 0.1697
# Epoch 2/10: Train Loss 0.1619, Learning Rate 0.0009
# Distance: 0.1710
# Epoch 3/10: Train Loss 0.1615, Learning Rate 0.0008
# Distance: 0.1723
# Epoch 4/10: Train Loss 0.1602, Learning Rate 0.0007
# Distance: 0.1704
# Epoch 5/10: Train Loss 0.1587, Learning Rate 0.0005
# Distance: 0.1700
# Epoch 6/10: Train Loss 0.1612, Learning Rate 0.0003
# Distance: 0.1686
# Epoch 7/10: Train Loss 0.1589, Learning Rate 0.0002
# Distance: 0.1692
# Epoch 8/10: Train Loss 0.1581, Learning Rate 0.0001
# Distance: 0.1706
# Epoch 9/10: Train Loss 0.1566, Learning Rate 0.0000
# Distance: 0.1706
# Epoch 10/10: Train Loss 0.1569, Learning Rate 0.0000
#                                                                Distance: 0.1709

In [ ]:
# Train Loss: 1.8752116763900066
# Val Loss: 0.17607808538845607
# Train Loss: 0.18389656677328306
# Val Loss: 0.17218721125807082
# Train Loss: 0.18046144164841751
# Val Loss: 0.16996091178485326
# Train Loss: 0.17665653187653113
# Val Loss: 0.17563885237489427
# Train Loss: 0.1673661090176681
# Val Loss: 0.17287699239594595
# Train Loss: 0.1738741408134329
# Val Loss: 0.17703682609966823
# Train Loss: 0.16847693868752184
# Val Loss: 0.17194960585662297
# Train Loss: 0.16885116619282756
# Val Loss: 0.16994784346648625
# Train Loss: 0.16396602397334986
# Val Loss: 0.17066440199102675
# Train Loss: 0.16824036682474203
# Val Loss: 0.1702498346567154

# SentimentRNN - Glove

## Utility functions for Glove

In [ ]:
GLOVE_EMBEDDING_SIZE = 100

In [ ]:
#Glove model--for some reason it doesnt load when importing from .py files so Im including it here

import torch
import torch.nn as nn


class GloVe(nn.Module):

    def __init__(self, vocab_size, embedding_size, x_max, alpha):
        super().__init__()
        self.weight = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_size,
            sparse=True
        )
        self.weight_tilde = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_size,
            sparse=True
        )
        self.bias = nn.Parameter(
            torch.randn(
                vocab_size,
                dtype=torch.float,
            )
        )
        self.bias_tilde = nn.Parameter(
            torch.randn(
                vocab_size,
                dtype=torch.float,
            )
        )
        self.weighting_func = lambda x: (x / x_max).float_power(alpha).clamp(0, 1)
    
    def forward(self, i, j, x):
        loss = torch.mul(self.weight(i), self.weight_tilde(j)).sum(dim=1)
        loss = (loss + self.bias[i] + self.bias_tilde[j] - x.log()).square()
        loss = torch.mul(self.weighting_func(x), loss).mean()
        return loss

In [ ]:
#!pip3 install --upgrade gensim --user

drive_path = '/content/drive/MyDrive/'

from pathlib import Path
import os
import argparse
import pickle

import torch
import yaml
from gensim.models.keyedvectors import KeyedVectors
import gensim.models

import h5py
import sys
import numpy as np 
sys.path.insert(0,'/content/drive/MyDrive/11785_Project/Programming/GloveCode')


def load_config():
    with open('/content/drive/MyDrive/11785_Project/Programming/GloveCode/config.yaml') as parameters:
            config_dict = yaml.safe_load(parameters)
    config = argparse.Namespace()
    for key, value in config_dict.items():
        setattr(config, key, value)
    return config


def make_vectors():
    config = load_config()
    with open(os.path.join('/content/drive/MyDrive/11785_Project/Programming/GloveCode', "vocab.pkl"), "rb") as f:
        vocab = pickle.load(f)

    model = GloVe(
        vocab_size=config.vocab_size,
        embedding_size=config.embedding_size,
        x_max=config.x_max,
        alpha=config.alpha
    )
    model.load_state_dict(torch.load('/content/drive/MyDrive/11785_Project/Programming/GloveCode/glove-output'))
    keyed_vectors = gensim.models.keyedvectors.KeyedVectors(vector_size=config.embedding_size)
    keyed_vectors.add_vectors(
        keys=[vocab.get_token(index) for index in range(len(vocab))],
        weights=(model.weight.weight.detach()
            + model.weight_tilde.weight.detach()).numpy()
    )

    return keyed_vectors, config.embedding_size, vocab


def get_word_embedding(word: str):
    keyed_vectors, emb_size, vocab=make_vectors()
    word_id = vocab[word]
    if word_id == -1:
        # print("Out of vocabulary word")
        return torch.zeros(emb_size, 1)

    word_vec = keyed_vectors.get_vector(word)
    word_vec = np.reshape(word_vec, (len(word_vec), 1))
    return word_vec

for w in ['burden', 'deliverable', 'domain']:
    embedding = get_word_embedding(w)

    print(f'{w} has embedding {embedding.shape}')

# _dir = '/content/drive/MyDrive/11785-project/word2vec/'
# vocab = torch.load(f'{word2vec_dir}vocab.pt')
# model = torch.load(f'{word2vec_dir}model.pt')

# # embedding from first model layer
# embeddings = list(model.parameters())[0]
# embeddings = embeddings.cpu().detach().numpy()

# # normalization
# norms = (embeddings * 2).sum(axis=1) * (1 / 2)
# norms = np.reshape(norms, (len(norms), 1))
# embeddings_norm = embeddings / norms
# embeddings_norm.shape

# def get_word_embedding(word: str):
#     word_id = vocab[word]
#     if word_id == 0:
#         # print("Out of vocabulary word")
#         return torch.zeros(embeddings_norm.shape[1], 1)

#     word_vec = embeddings_norm[word_id]
#     word_vec = np.reshape(word_vec, (len(word_vec), 1))
#     return word_vec

# for w in ['burden', 'deliverable', 'domain']:
#     embedding = get_word_embedding(w)

#     print(f'{w} has embedding {embedding.shape}')

burden has embedding (100, 1)
deliverable has embedding (100, 1)
domain has embedding (100, 1)


## Dataloader

In [ ]:
class FiQaSamplesWord2Vec(torch.utils.data.Dataset):

  def __init__(self, partition, sentences, sentiment_scores, get_word_embedding, max_len):
    self.sentences = sentences
    self.sentiment_scores = sentiment_scores
    self.model = get_word_embedding
    assert len(sentences) == len(sentiment_scores)
    self.length = len(self.sentences)
    self.max_len = max_len
  
  def __len__(self):
    return self.length
  
  def __getitem__(self, index):
    sentence = self.sentences[index]
    score = self.sentiment_scores[index]
    tokens = sentence
    # Tokens is a list of strings
    embedding = np.array([np.array(self.model(token)) for token in sentence.split()])
    return torch.tensor(embedding).squeeze(), torch.tensor(score, dtype = torch.float)

  def collate_fn(batch):
    batch_x = [emb for emb, y in batch]
    batch_y = [y for emb, y in batch]

    batch_x_pad = pad_sequence(batch_x, batch_first = True) 
    lengths_x = [x.shape[0] for x in batch_x]

    return batch_x_pad, lengths_x, torch.tensor(batch_y)

In [ ]:
batch_size = 32
train_sentences, train_scores = df_train.sentence.to_list(), df_train.sentiment_score.to_list()
val_sentences, val_scores = df_val.sentence.to_list(), df_val.sentiment_score.to_list()

train_data = FiQaSamplesWord2Vec('train', train_sentences, train_scores, get_word_embedding, 128)
val_data = FiQaSamplesWord2Vec('dev', val_sentences, val_scores, get_word_embedding, 128)
# test_data = FiQaSamples('test_order.csv')

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=1, collate_fn=FiQaSamplesWord2Vec.collate_fn) # TODO: Define the train loader. Remember to pass in a parameter (function) for the collate_fn argument 
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=1, collate_fn=FiQaSamplesWord2Vec.collate_fn) # TODO: Define the val loader. Remember to pass in a parameter (function) for the collate_fn argument 
# test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=1, collate_fn=LibriSamplesTest.collate_fn ) # TODO: Define the test loader. Remember to pass in a parameter (function) for the collate_fn argument 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Batch size: ", batch_size)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
# print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))
print("Device: ", device)

Batch size:  32
Train dataset samples = 911, batches = 29
Val dataset samples = 200, batches = 7
Device:  cuda


## Model Architecture

In [ ]:
# https://www.kaggle.com/code/arunmohan003/sentiment-analysis-using-lstm-pytorch/notebook
class SentimentRNN(nn.Module):
    def __init__(self):
        super(SentimentRNN,self).__init__()
        self.hidden_size = 128
        self.lstm = nn.LSTM(input_size=GLOVE_EMBEDDING_SIZE, hidden_size=self.hidden_size, num_layers=2, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, 1)
        
    def forward(self, x, lx):
        # x is a list of vector (num of tokens, emb_dim - hyperparam)
        batch_size = x.shape[0]
        packed_input = pack_padded_sequence(x, lx, batch_first = True, enforce_sorted=False) 
        output, (hidden_state, cell_state) = self.lstm(packed_input) 
        out, lengths  = pad_packed_sequence(output, batch_first = True)
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.dropout(out)
        out = self.fc(out)

        out = out.view(batch_size, -1)
        out = out[:, -1] # get last batch of labels
        return out, lengths


## Training and validation functions

In [ ]:
def run_train(model, train_loader, batch_size, optimizer, criterion, scheduler, epoch, epochs):
  torch.cuda.empty_cache()
  model.train()
  batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

  num_correct = 0
  total_loss = 0

  for i, (x, lx, y) in enumerate(train_loader):
      torch.cuda.empty_cache() # Use this often
      optimizer.zero_grad()

      x = x.to(device)
      y = y.to(device)

      outputs, lengths = model(x, lx)
      loss = criterion(outputs, y)
      total_loss += float(loss)
      batch_bar.set_postfix(
          loss="{:.04f}".format(float(total_loss / (i + 1))),
          lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
      
      # Another couple things you need for FP16. 
      loss.backward()
      optimizer.step()


      batch_bar.update() # Update tqdm bar
  scheduler.step() # We told scheduler T_max that we'd call step() (len(train_loader) * epochs) many times.
  batch_bar.close() # You need this to close the tqdm bar

  print("Epoch {}/{}: Train Loss {:.04f}, Learning Rate {:.04f}".format(
      epoch + 1,
      epochs,
      float(total_loss / len(train_loader)),
      float(optimizer.param_groups[0]['lr'])))
  


def run_validation(model, val_loader):
  torch.cuda.empty_cache() # Use this often
  model.eval()
  batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
  total_loss = 0
  with torch.no_grad():
      for i, (x, lx, y) in enumerate(val_loader):
          torch.cuda.empty_cache() # Use this often
          x = x.to(device)
          y = y.to(device)

          outputs, lengths = model(x, lx)
          loss = criterion(outputs, y)
          total_loss += loss
          batch_bar.set_postfix(dist="{:.04f}".format(total_loss / (i + 1)))

          batch_bar.update()
          
      batch_bar.close()


      print("Distance: {:.04f}".format(total_loss / len(val_loader)))

In [ ]:
lr = 1e-3
epochs = 10

model = SentimentRNN()
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
criterion = nn.MSELoss()

## Lets train!

In [ ]:
for epoch in range(epochs):
    run_train(model, train_loader, batch_size, optimizer, criterion, scheduler, epoch, epochs)
    run_validation(model, val_loader)


Epoch 1/10: Train Loss 0.1862, Learning Rate 0.0010


Distance: 0.1858


Epoch 2/10: Train Loss 0.1776, Learning Rate 0.0009


Distance: 0.1785


Epoch 3/10: Train Loss 0.1731, Learning Rate 0.0008


...

Distance: 0.1743


Epoch 9/10: Train Loss 0.1609, Learning Rate 0.0000


Distance: 0.1743


Epoch 10/10: Train Loss 0.1619, Learning Rate 0.0000


Distance: 0.1738
